# 08. Pairing Cloud and non-Cloud Data

## Accessing and harmonizing data located within and outside of the NASA Earthdata Cloud

## Summary

This tutorial will combine several workflow steps and components from the previous days, demonstrating the process of using the geolocation of data available outside of the Earthdata Cloud to then access coincident variables of cloud-accessible data. This may be a common use case as NASA Earthdata continues to migrate to the cloud, producing a "hybrid" data archive across Amazon Web Services (AWS) and original on-premise data storage systems. Additionally, you may also want to combine field measurements with remote sensing data available on the Earthdata Cloud.

This specific example explores the pairing of the ICESat-2 [ATL07 Sea Ice Height](https://nsidc.org/data/atl07) data product, currently (as of November 2021) available publicly via direct download at the NSIDC DAAC, along with Sea Surface Temperature (SST) from the GHRSST MODIS L2 dataset ([MODIS_A-JPL-L2P-v2019.0](https://cmr.earthdata.nasa.gov/search/concepts/C1693233348-PODAAC.html)) available from PO.DAAC on the Earthdata Cloud. 

The use case we're looking at today centers over an area north of Greenland for a single day in June, where a melt pond was observed using the [NASA OpenAltimetry](https://openaltimetry.org/data/icesat2/?start_date=2019-06-22&annoId=180) application. Melt ponds are an important feature of Arctic sea ice dynamics, leading to an decrease in sea ice albedo and other changes in heat balance. Many NASA Earthdata datasets produce variables including sea ice albedo, sea surface temperature, air temperature, and sea ice height, which can be used to better understand these dynamics. This notebook walks through the discovery, subsetting, and accessing of multiple data products coincident over an Arctic Ocean melt pond region, demonstrating how to obtain and harmonize coincident remote sensing data.

### Objectives

1. Practice skills searching for data in CMR, determining storage location and granule coverage across two datasets over an area of interest
2. Download data from an on-premise storage system to our cloud environment
3. Read in 1-dimensional trajectory data (ICESat-2 ATL07) into xarray and perform attribute conversions
4. Select and read in sea surface temperature (SST) data (MODIS_A-JPL-L2P-v2019.0) from the Earthdata Cloud into xarray
5. Extract and resample coincident SST data based on ICESat-2 geolocation

---

## Import packages

In [ ]:
#

## Determine storage location of datasets of interest

First, let's see whether our datasets of interest reside in the Earthdata Cloud or whether they reside on premise, or "on prem" at a local data center.

We are building off of the CMR introductory tutorial, beginning with a collection search.

In [ ]:
#

We want to search by collection to inspect the access and service options that exist:

In [ ]:
#

In the CMR introduction tutorial, we explored cloud-hosted collections from different DAAC providers, and identified the CMR concept-id for a given dataset id (also referred to as a short_name). Here we'll start with two datasets that we want to explore over a coincident area and time:

In [ ]:
#

Like in the intro tutorial, we're going to first determine what concept-ids are returned for the MODIS dataset. First, retrieve collection results based on the MODIS `short_name`:

In [ ]:
#

For each collection result, print out the CMR concept-id and version:

In [ ]:
#

Two collections are returned, both at version 2019.0. We can see from the suffix of the id that one is associated with "POCLOUD" versus "PODAAC". That gives us a clue in terms of where the data are hosted, but we can also use the `cloud_hosted` parameter set to True to confirm.

In [ ]:
#

In [ ]:
#

We will save this concept-id to use later on when we access the data granules.

In [ ]:
#

Now we will try our ICESat-2 dataset to see what id's are returned for a given dataset name.

In [ ]:
#

In [ ]:
#

Two separate datasets exist in the CMR, one at version 3 and one at version 4. Let's see if these are `cloud_hosted`:

In [ ]:
#

In [ ]:
#

When set to `False`, we get our collections back. We have now determined that we have a copy of the MODIS dataset in the cloud, whereas the ICESat-2 dataset (both versions) remains "on premise", residing in a local data center. 

Save the ATL07 concept ID:

In [ ]:
#

## Specify time range and area of interest 

We are going to focus on getting data for an area north of Greenland for a single day in June.

These `bounding_box` and `temporal` variables will be used for data search, subset, and access below

In [ ]:
#

Perform a granule search over our time and area of interest. How many granules are returned?

In [ ]:
#

In [ ]:
#

Print the file names, size, and links:

In [ ]:
#

## Download ICESat-2 ATL07 granule

Although several services are supported for ICESat-2 data, we are demonstrating direct access through the "on-prem" file system at NSIDC for simplicity. Other services include:
* [icepyx](https://icepyx.readthedocs.io/en/latest/)
    * From the icepyx documentation: "icepyx is both a software library and a community composed of ICESat-2 data users, developers, and the scientific community. We are working together to develop a shared library of resources - including existing resources, new code, tutorials, and use-cases/examples - that simplify the process of querying, obtaining, analyzing, and manipulating ICESat-2 datasets to enable scientific discovery."
* [NSIDC DAAC Data Access and Service API](https://nsidc.org/support/how/how-do-i-programmatically-request-data-services#API) 
    * The API provided by the NSIDC DAAC allows you to access data programmatically using specific temporal and spatial filters. The same subsetting, reformatting, and reprojection services available on select data sets through NASA Earthdata Search can also be applied using this API.
* [IceFlow](https://github.com/nsidc/NSIDC-Data-Tutorials/tree/main/notebooks/iceflow)
    * The IceFlow python library simplifies accessing and combining data from several of NASA's cryospheric altimetry missions, including ICESat/GLAS, Operation IceBridge, and ICESat-2. In particular, IceFlow harmonizes the various file formats and georeferencing parameters across several of the missions' data sets, allowing you to analyze data across the multi-decadal time series.

We've found 2 granules.  We'll download the first one and write it to a file with the same name as the `producer_granule_id`.

We need the url for the granule as well.  This is `href` links we printed out above.

In [ ]:
#

To retrieve the granule data, we use the `requests.get()` method, which will utilize the .netrc file on the backend to authenticate the request against Earthdata Login.  

In [ ]:
#

The response returned by requests has the same structure as all the other responses: a header and contents.  The header information has information about the response, including the size of the data we downloaded in bytes. 

In [ ]:
#

The contents needs to be saved to a file.  To keep the directory clean, we will create a `downloads` directory to store the file.  We can use a shell command to do this or use the `makedirs` method from the `os` package. 

In [ ]:
#

You should see a `downloads` directory in the file browser.

To write the data to a file, we use `open` to open a file.  We need to specify that the file is open for writing by using the _write-mode_ `w`.  We also need to specify that we want to write bytes by setting the _binary-mode_ `b`.  This is important because the response contents are bytes.  The default mode for `open` is `text-mode`. So make sure you use `b`.

We'll use the `with` statement _context-manager_ to open the file, write the contents of the response, and then close the file.  Once the data in `r.content` is written sucessfully to the file, or if there is an error, the file is closed by the _context-manager_.

We also need to prepend the `downloads` path to the filename.  We do this using `Path` from the `pathlib` package in the standard library.

In [ ]:
#

In [ ]:
#

Check to make sure it is downloaded.

In [ ]:
#

In [ ]:
#

### Read file into xarray

`ATL07-01_20190622055317_12980301_004_01.h5` is an HDF5 file.  `xarray` can open this but you need to tell it which group to read the data from.  In this case we read the sea ice segment height data for ground-track 1 left-beam.

This code block performs the following operations:
* Extracts the `height_segment_height` variable from the `heights` group, along with the dimension variables contained in the higher level `sea_ice_segments` group
* Convert attributes from bytestrings to strings
* Drops the HDF attr DIMENSION_LIST 
* Sets _FillValue to NaN

In [ ]:
#

## Determine the GHRSST MODIS L2 granules returned from our time and area of interest

In [ ]:
#

In [ ]:
#

## Load data into xarray via S3 direct access

Our CMR granule search returned 14 files for our time and area of interest.  However, not all granules will be suitable for analysis.

I've identified the image with granule id G1956158784-POCLOUD as a good candidate, this is the 9th granule.  In this image, our area of interest is close to nadir.  This means that the instantaneous field of view over the area of interest cover a smaller area than at the edge of the image.

We can use `pprint` to look at the metadata.

We are looking for the link for _direct download access via s3_.  This is a url but with a prefix `s3://`.  This happens to be the first href link in the metadata.  

In [ ]:
#

For a single granule we can cut and paste the s3 link.  If we have several granules, the s3 links can be extracted with some simple code.

In [ ]:
#

### Get S3 credentials
As with the previous S3 download tutorials we need credentials to access data from s3: access keys and tokens.

In [ ]:
#

Essentially, what we are doing in this step is to "mount" the s3 bucket as a file system.  This allows us to treat the S3 bucket in a similar way to a local file system.  We can list data using `ls`, copy data, and write data - assuming we have the permissions to do this.

In [ ]:
#

### Open a s3 file

Now we have the `S3FileSystem` set up, we can access the granule.  `xarray` cannot open a `S3File` directly, so we use the `open` method for the `S3FileSystem` to open the granule using the endpoint url we extracted from the metadata.  We also have to set the `mode='rb'`.  This opens the granule in read-only mode and in byte-mode.  Byte-mode is important.  By default, `open` opens a file as text - in this case it would just be a string of characters - and `xarray` doesn't know what to do with that.

We then pass the `S3File` object `f` to `xarray.open_dataset`.  For this dataset, we also have to set `decode_cf=False`.  This switch tells `xarray` not to use information contained in variable attributes to generate human readable coordinate variables.  Normally, this should work for netcdf files but for this particular cloud-hosted dataset, variable attribute data is not in the form expected by `xarray`.  We'll fix this.

In [ ]:
#

If you click on the `Show/Hide Attributes` icon (the first document-like icon to the right of coordinate variable metadata) you can see that attributes are one-element arrays containing bytestrings.

In [ ]:
#

To fix this, we need to extract array elements as scalars, and convert those scalars from bytestrings to strings.  I use the `decode` method to do this.  The bytestrings are encoded as utf-8, which is a unicode character format.  This is the default encoding for `decode` but I've included it as an argument to be explicit.

Not all attributes are bytestrings.  Some are `floats`.  Take a look at `_FillValue`, and `valid_min` and `valid_max`.  To avoid an error, I use the `isinstance` function to check if the value of an attributes is type `bytes` - a bytestring.  If it is, I decode it.  If not, I just extract the scalar and do nothing else.

I also fix the global attributes.

In [ ]:
#

With this done, we can use the `xarray` function `decode_cf` to convert the attributes. 

In [ ]:
#

In [ ]:
#

Let's make a quick plot to take a look at the `sea_surface_temperature` variable.

In [ ]:
#

## Extract coincident SST data based on ICESat-2 transect

### Interpolating MODIS data onto ICESat-2 points
To easily interpolate our MODIS sea surface temperature data to ICESat-2 points, we can use the `pyresample` library. The `radius_of_influence` parameter determines maximum radius to look for nearest neighbor interpolation.

In [ ]:
#

In [ ]:
#

In [ ]:
#

### Plot ICESat-2 height values over MODIS Sea Surface Temperature

In [ ]:
#

## Ancillary Material: Find suitable granule

In [ ]:
#

In [ ]:
#